In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import csv
import os
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import spacy
from spacy.tokens import Token
import pickle as pkl
import string
from fuzzywuzzy import fuzz




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movies-dataset/movies.csv
/kaggle/input/ratings-dataset/ratings.csv


In [2]:
movies = pd.read_csv("/kaggle/input/movies-dataset/movies.csv")
print(movies.head())
print(movies.shape)

   movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)
(84661, 2)


In [3]:
ratings = pd.read_csv("/kaggle/input/ratings-dataset/ratings.csv")
ratings_half = ratings.head(len(ratings) // 8)

In [4]:
movies_users = ratings_half.pivot_table(index='userId',columns="movieId", values="rating", aggfunc='mean').fillna(0)


In [5]:
compressed_movies_users = csr_matrix(movies_users)

In [6]:

model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
model.fit(compressed_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [7]:
def recommander_function(userInd,data,n,model):
    distance, indices  = model.kneighbors(data[userInd], n_neighbors=n)
    print(distance)
    print(indices)
    indices = indices[0]
    best_movies_ids = []
    for i in indices :
        similar_users = movies_users.iloc[i]
        top_movies = similar_users[similar_users == 5]
        best_movies_ids += top_movies.index.tolist()
    best_movies_ids = list(set(best_movies_ids))
    print(best_movies_ids)
    print(movies.loc[best_movies_ids])

In [8]:
recommander_function(0, compressed_movies_users, 5,model)

[[0.         0.06774334 0.07773768 0.09498165 0.11751676]]
[[    0 15758 12001 22235  5252]]
[736, 648, 780, 62, 95]
     movieId                                              title
736      749                    Man from Down Under, The (1943)
648      656                                       Eddie (1996)
780      793  My Life and Times With Antonin Artaud (En comp...
62        63  Don't Be a Menace to South Central While Drink...
95        97                            Hate (Haine, La) (1995)


In [9]:
filename = "/kaggle/working/NearestNeighborsModelTest"
pickle = pkl.dump(model, open(filename, 'wb'))